# Avaliação de ações

Este notebook tem como objetivo a criação de dataframes para uso no Power BI, visando a avaliação e acompanhamento das negociações de ativos em carteira

#### Bibliotecas

Fazendo uso da biblioteca pandas_datareader traremos as cotações de cada ativo em carteir, bem como ativos que replicam indices(IVVB11, BOVA11) para te-lo como benchmark

In [1]:
import pandas_datareader.data as web
import pandas as pd

As informações de movimentação em carteira serão coletadas apartir de uma base xlsx, contendo esta as informações de:

Ticker;
Valor Cotação;
Qtd; 
Vl. Operação; 
Tipo Operação; 
Data; 
Produto

In [2]:
movimentacoes = pd.read_excel(r'C:\Users\Desktop\Desktop\Power BI - Portifolio\04 - Base Excel\Movimentacoes_carteira.xlsx')

#### Composição da Carteira

A composição da carteira se dará por meio de um group by como forma de consolidar todas as compras e vendas por Ticker 

In [3]:
volume_ativos =  movimentacoes[['TICKER','Qtd']].groupby(['TICKER']).sum()
aporte =  movimentacoes[['TICKER','Valor']].groupby(['TICKER']).sum()
preco_medio_ativos = movimentacoes[['TICKER','Valor']].groupby(['TICKER']).mean()
tipo_produto = movimentacoes[['TICKER', 'Produto']].drop_duplicates()

carteira = preco_medio_ativos.merge(volume_ativos, how='inner', on='TICKER')
carteira = carteira.merge(tipo_produto, how='inner', on='TICKER')
carteira = carteira.merge(aporte, how='inner', on='TICKER')
carteira.rename(columns = {'TICKER'   : 'Ticker',
                           'Valor_x'  : 'Preco_medio',
                           'Qtd'      : 'Posicao',
                           'Produto'  : 'Classes',
                           'Valor_y'  : 'Aporte'},  inplace = True)
carteira

,Ticker,Preco_medio,Posicao,Classes,Aporte
0,AAPL34,68.08,7,BDR,68.08
1,ALPA3,31.00,10,AÇÕES,31.00
2,AMZO34,108.84,4,BDR,108.84
3,ARZZ3,73.41,4,AÇÕES,73.41
4,BBAS3,30.02,10,AÇÕES,30.02
5,BBSE3,28.83,13,AÇÕES,28.83
6,BCFF11,89.45,15,FUNDOS,89.45
7,BOAC34,52.82,9,BDR,52.82
8,CASH3,27.31,12,AÇÕES,27.31
9,CMIG3,14.31,22,AÇÕES,14.31


 Para iniciar as buscas de cotações inicialmente será listado todos os ativos em carteira

In [4]:
list_ticker = list(carteira.Ticker.unique())

### Dataset Histórico

Neste momento será buscado todas as cotações históricas dos ativos que compõem a carteira, como meio de observar ao track record de cada ativo.

In [5]:
column_names = ['Date','Open','High','Low','Close','Adj Close','Volume','Ticker']
ticker_historical = pd.DataFrame(columns = column_names)

for i in list_ticker:
    t = i.strip()+'.SA'
    historical = pd.DataFrame(data=web.get_data_yahoo(t))
    historical['Ticker'] = t
    ticker_historical = ticker_historical.append(historical)
    



Apenas algumas formatações no dataframe para ficar melhor a utilização no Power BI

In [6]:
ticker_historical.to_csv('temp.csv')
historico = pd.read_csv('temp.csv')

historico['Date'] = historico['Unnamed: 0']
historico = historico.drop(columns=['Unnamed: 0'])


### Benchmark

BOVA11 é o ETF que replica o indice Bovespa

In [7]:
benchmark = 'BOVA11.SA'
column_names = ['Date','Open','High','Low','Close','Adj Close','Volume','Ticker']
benchmark_historical = pd.DataFrame(columns = column_names)
t = benchmark.strip()
historical = pd.DataFrame(data=web.get_data_yahoo(t))
historical['Ticker'] = t
benchmark_historical = benchmark_historical.append(historical)
    
benchmark_historical.to_csv('temp.csv')
bovespa = pd.read_csv('temp.csv')


bovespa['Date'] = bovespa['Unnamed: 0']
bovespa = bovespa.drop(columns=['Unnamed: 0'])




In [8]:
benchmark = 'IVVB11.SA'
column_names = ['Date','Open','High','Low','Close','Adj Close','Volume','Ticker']
benchmark_historical = pd.DataFrame(columns = column_names)
t = benchmark.strip()
historical = pd.DataFrame(data=web.get_data_yahoo(t))
historical['Ticker'] = t
benchmark_historical = benchmark_historical.append(historical)
    
benchmark_historical.to_csv('temp.csv')
ivvb = pd.read_csv('temp.csv')


ivvb['Date'] = ivvb['Unnamed: 0']
ivvb = ivvb.drop(columns=['Unnamed: 0'])

